In [72]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# model.save("app/models/all-MiniLM-L6-v2")

In [1]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, MilvusClient, utility
from app.models.bge_m3 import BGEM3Model
model = BGEM3Model()

In [4]:
# Define the collection name
connections.connect( alias="default", host='localhost', port='19530')

COLLECTION_NAME = "semantic_search"
fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=50000)
    ]
schema = CollectionSchema(fields, "Semantic Search Collection")
index_params = {"index_type": "IVF_FLAT", "metric_type": "L2", "params": {"nlist": 128}}
collection = Collection(COLLECTION_NAME,schema)
collection.create_index("embedding", index_params)
collection.load()

In [13]:
collection.load()

In [14]:
data = [
   {
    "embedding": model.encode("hello world 1"),
    "text": "hello world 1"
    }
]
collection.insert(data)

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 456645693874634753, success count: 1, err count: 0

In [ ]:
collection

<Collection>:
-------------
<name>: semantic_search
<description>: Semantic Search Collection
<schema>: {'auto_id': True, 'description': 'Semantic Search Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 50000}}], 'enable_dynamic_field': False}

In [ ]:
client = MilvusClient(host="localhost", port="19530")
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    index_type="IVF_FLAT",
    metric_type="IP",
    params={"nlist": 128}
)
schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=25600)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=384)

client.create_collection(collection_name="retrieval", schema=schema,index_params= index_params)

In [ ]:
# connections.connect( host="localhost", port="19530")
# connections.list_connections()

In [ ]:
client.insert(collection_name="retrieval", data=data)

NameError: name 'client' is not defined

In [20]:
emb  = model.encode("hello world 3")

In [5]:
client.query(
    collection_name="retrieval",
    # filter="text like \"hello\"",
    output_fields=["text"],
    limit=3
)

NameError: name 'client' is not defined

In [8]:
# collection.search(data[0]["embedding"], "embedding", limit=3)
collection

<Collection>:
-------------
<name>: semantic_search
<description>: Semantic Search Collection
<schema>: {'auto_id': True, 'description': 'Semantic Search Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 50000}}], 'enable_dynamic_field': False}

In [21]:
search_params = {"metric_type": "L2", "params": {"nprobe": 16}}
tmp = collection.search( data=[emb],  anns_field="embedding", limit=3, param=search_params, output_fields=["text"])
print(tmp)
# {"results": [{"text": hit['entity']["text"]} for hit in tmp[0]]}

data: ['["id: 456645637313086904, distance: 0.20131678879261017, entity: {\'text\': \'hello world 1\'}", "id: 456639586658681161, distance: 0.3006055951118469, entity: {\'text\': \'hello world\'}", "id: 456639586658681163, distance: 0.3006055951118469, entity: {\'text\': \'hello world\'}"]']


In [44]:
tmp[0][0]

id: 456645637313086904, distance: 0.20131678879261017, entity: {'text': 'hello world 1'}

In [41]:
tmp[0][0].entity.text

'hello world 1'

In [42]:
[{"text": hit.entity.text} for hit in tmp[0]]

[{'text': 'hello world 1'}, {'text': 'hello world'}, {'text': 'hello world'}]

In [ ]:
search_params = {"metric_type": "IP", "params": {}}
print('-'*100)
results = client.search(
    collection_name="retrieval",
    data=[list(emb)],
    limit=3,
    output_fields=["text"],
    search_params=search_params
)

2025-03-13 20:23:20,540 [ERROR][handler]: Unexpected error: [search], GrpcHandler.search() got multiple values for argument 'param', <Time: {'RPC start': '2025-03-13 20:23:20.540812', 'Exception': '2025-03-13 20:23:20.540833'}> (decorators.py:158)
2025-03-13 20:23:20,541 [ERROR][search]: Failed to search collection: retrieval (milvus_client.py:415)


----------------------------------------------------------------------------------------------------


MilvusException: <MilvusException: (code=1, message=Unexpected error, message=<GrpcHandler.search() got multiple values for argument 'param'>)>